<a href="https://colab.research.google.com/github/surensnyper/ys_project_001/blob/main/EMF_Model_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.youtube.com/watch?v=f-86-HcYYi8
# https://www.youtube.com/c/ChrisMcCormickAI/playlists
from google.colab import files
uploaded = files.upload()

Saving Test dataset.xlsx to Test dataset.xlsx
Saving Train dataset.xlsx to Train dataset.xlsx


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 75.0 MB/s 
     |████████████████████████████████| 895 kB 67.6 MB/s 
     |████████████████████████████████| 3.3 MB 54.5 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [ ]:
import pandas as pd
train_df = pd.read_excel('Train dataset.xlsx')
test_df = pd.read_excel('Test dataset.xlsx')

In [ ]:
train_df.columns

Index(['Paper ', 'Abstract', 'Target', 'Method In-Vivo', 'Method In-Vitro',
       'Method Others'],
      dtype='object')

In [ ]:
train_df = train_df[['Abstract', 'Method In-Vivo', 'Method In-Vitro', 'Method Others',]]

In [ ]:
train_df.head()

,Abstract,Method In-Vivo,Method In-Vitro,Method Others
0,It has been suggested that electromagnetic fie...,1,0,0
1,The intracranial 9L tumor model was used to de...,1,0,0
2,The purpose of this study was to determine whe...,1,0,0
3,The purpose of this study was to determine if ...,1,0,0
4,Our goal was to investigate effects of long-te...,1,0,0


In [ ]:
target_list = ['Method In-Vivo', 'Method In-Vitro', 'Method Others']

In [ ]:
# hyperparameters
MAX_LEN = 256 # Embedded word vector's size (256 vectors representing word features)
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-05

In [ ]:
# Tokenization
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
tokenized = train_df['Abstract'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized

0      [101, 2009, 2038, 2042, 4081, 2008, 17225, 424...
1      [101, 1996, 26721, 26775, 27532, 1023, 2140, 1...
2      [101, 1996, 3800, 1997, 2023, 2817, 2001, 2000...
3      [101, 1996, 3800, 1997, 2023, 2817, 2001, 2000...
4      [101, 2256, 3125, 2001, 2000, 8556, 3896, 1997...
                             ...                        
338    [101, 2270, 5936, 2058, 2825, 15316, 3896, 199...
339    [101, 1996, 6923, 2224, 1997, 4684, 11640, 132...
340    [101, 3728, 1010, 2116, 2913, 2031, 2042, 3344...
341    [101, 7863, 1024, 1996, 6614, 1997, 2023, 2817...
342    [101, 7863, 1024, 1996, 3896, 1997, 17225, 824...
Name: Abstract, Length: 343, dtype: object

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = self.df['Abstract']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
train_size = 0.8
train_df = train_df.sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = train_df.drop(train_df.index).reset_index(drop=True)

In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(
    valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
device

device(type='cuda')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 3)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train_model(
    n_epochs, 
    training_loader, 
    validation_loader, 
    model, 
    optimizer, 
    checkpoint_path, 
    best_model_path
):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for index, batch in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        input_ids = batch['input_ids'].to(device, dtype = torch.long)
        attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)
        outputs = model(input_ids, attention_mask, token_type_ids)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1/(index+1)) * (loss.item()-train_loss))
        #print('after loss data in training', loss.item(), train_loss)
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
      for index, batch in enumerate(validation_loader):
            input_ids = batch['input_ids'].to(device, dtype = torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.float)
            outputs = model(input_ids, attention_mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1/(index+1)) * (loss.item()-valid_loss))
            
      print('############# Epoch {}: Validation End     #############'.format(epoch))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        
        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)
      
    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:
ckpt_path = "/content/drive/MyDrive/Colab Notebooks/EQA_dataset/EMF/curr_ckpt"
best_model_path = "/content/drive/MyDrive/Colab Notebooks/EQA_dataset/EMF/best_model.pt"

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epoch 3: Validation End     #############
############# Epoch 3  Done   #############

############# Epoch 4: Training Start   #############
############# Epoch 4: Training End     #############
############# Epoch 4: Validation Start   #############
############# Epoch 4: Validation End    

In [ ]:
torch.save(trained_model, 'model.pt')

In [ ]:
# Testing

for i in range(0, 114):
  
  print([i], end='  ')
  example = test_df['Abstract'][i]
  # example

  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )

  model.eval()
  with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype = torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype = torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    # print(final_output)
    print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])

[0]  Method In-Vivo
[1]  Method In-Vivo
[2]  Method In-Vivo
[3]  Method In-Vivo
[4]  Method In-Vivo
[5]  Method In-Vivo
[6]  Method In-Vivo
[7]  Method In-Vivo
[8]  Method In-Vivo
[9]  Method In-Vitro
[10]  Method In-Vivo
[11]  Method In-Vivo
[12]  Method In-Vivo
[13]  Method In-Vivo
[14]  Method Others
[15]  Method In-Vitro
[16]  Method In-Vivo
[17]  Method In-Vivo
[18]  Method In-Vivo
[19]  Method In-Vivo
[20]  Method In-Vivo
[21]  Method In-Vivo
[22]  Method In-Vivo
[23]  Method In-Vivo
[24]  Method In-Vivo
[25]  Method In-Vivo
[26]  Method In-Vivo
[27]  Method In-Vivo
[28]  Method In-Vitro
[29]  Method In-Vivo
[30]  Method In-Vitro
[31]  Method In-Vitro
[32]  Method In-Vivo
[33]  Method In-Vivo
[34]  Method In-Vivo
[35]  Method In-Vivo
[36]  Method In-Vitro
[37]  Method Others
[38]  Method In-Vivo
[39]  Method In-Vitro
[40]  Method In-Vitro
[41]  Method In-Vitro
[42]  Method In-Vivo
[43]  Method In-Vivo
[44]  Method In-Vivo
[45]  Method In-Vivo
[46]  Method In-Vivo
[47]  Method In-

In [ ]:
train_df.columns

Index(['Abstract', 'Method In-Vivo', 'Method In-Vitro', 'Method Others'], dtype='object')